In [1]:
import anthropic
from anthropic.types.messages.batch_create_params import Request
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
import pandas as pd
import numpy as np
import re
from collections import Counter
import random
import math
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import classification_report

In [2]:
def extract_first_number(s):
    """
    Extracts the first integer from a string, ignoring decimals.

    Args:
        s: The input string.

    Returns:
        The first integer found in the string, or None if no integer is found.
    """
    match = re.search(r'\d+', s)  # Matches only integers
    if match:
        return int(match.group())
    else:
        return None


## Find modes ###
def find_mode(data):
    """
    Finds the mode(s) of a list or pandas Series.
    
    Args:
        data: A list or pandas Series of values.
    
    Returns:
        A single mode if there are multiple modes;
        otherwise returns the mode(s) of the input data.
        Returns an empty list if the input is empty.
    """
    counts = Counter(data)
    max_count = max(counts.values())
    modes = [key for key, value in counts.items() if value == max_count]
    
    # If there are multiple modes, choose one randomly
    if len(modes) > 1:
        return random.choice(modes)
    
    return modes[0]


In [10]:
path = 'C:/Users/
role = "You are a helpful rater in science education. You should rate students' responses based on the scoring guide and only return the scores.\n"

item='VR1'
scoring_prompt = pd.read_csv(path+'scoring prompts.csv')
scoring_prompt = scoring_prompt.loc[scoring_prompt.Item == item,'Prompt'].values[0]
resp = pd.read_excel(path+'All Graded Data from LPS3 2022_ Undergraduates.xlsx',sheet_name=item)
resp.replace({'Final': {'0a': 0}},inplace=True)
resp.replace({'Final': {'0b': 0}},inplace=True)
resp['Final']=resp['Final'].astype(int)

nrep=5

C:\Users\mingf\AppData\Local\Temp\ipykernel_59608\1943850481.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resp.replace({'Final': {'0b': 0}},inplace=True)


In [4]:
cl_client = anthropic.Anthropic(api_key="input your own key")

In [ ]:
# Temperature =0

In [11]:
requests = []
tempr = 0
for index, row in resp.iterrows():
    #print(index)
    scoring_prompt_resp = scoring_prompt+'\n'+'RESPONSE:'+str(row[item])+'. ->'
    requests.append(
        Request(
            custom_id="item_"+item+"resp_"+str(index),
            params=MessageCreateParamsNonStreaming(
            model="claude-3-7-sonnet-20250219",
            temperature=tempr,
            max_tokens=1024,
            system=role,
            messages=[
                            {"role": "user", "content": scoring_prompt_resp}]
                )
            )
        )
            

In [12]:
batch1 = cl_client.messages.batches.create(requests=requests)
batch2 = cl_client.messages.batches.create(requests=requests)
batch3 = cl_client.messages.batches.create(requests=requests)
batch4 = cl_client.messages.batches.create(requests=requests)
batch5 = cl_client.messages.batches.create(requests=requests)

In [13]:
rrsspp1 = cl_client.messages.batches.results(batch1.id)
rrsspp2 = cl_client.messages.batches.results(batch2.id)
rrsspp3 = cl_client.messages.batches.results(batch3.id)
rrsspp4 = cl_client.messages.batches.results(batch4.id)
rrsspp5 = cl_client.messages.batches.results(batch5.id)

output0 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in rrsspp1]
output1 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in rrsspp2]
output2 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in rrsspp3]
output3 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in rrsspp4]
output4 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in rrsspp5]

In [14]:
scD_0 = pd.DataFrame({'output_0':output0, 'output_1':output1,'output_2':output2,'output_3':output3,'output_4':output4})

scD_0 = pd.concat([scD_0,scD_0.apply(lambda col: [extract_first_number(i) for i in col]).\
rename(columns=dict(zip(['output_'+str(i )for i in range(nrep)],['score_'+str(i )for i in range(nrep)])))], axis=1)

scD_0['mode'] = scD_0.iloc[:,nrep:(2*nrep)].apply(find_mode,axis=1)

scD_0.to_csv(path+'results/'+item+'_claude_tempr'+str(0)+'.csv')

In [18]:
scD_t = pd.concat([scD_0,resp.Final],axis=1).dropna()

In [19]:
print(f"QWK: {cohen_kappa_score(scD_t['mode'],scD_t['Final'],weights='quadratic')}")
print(classification_report(scD_t['mode'],scD_t['Final']))

QWK: 0.6220900084604678
              precision    recall  f1-score   support

           0       0.89      0.94      0.91        66
           1       0.94      0.68      0.79       297
           2       0.50      0.75      0.60        91
           3       0.00      0.00      0.00         0

    accuracy                           0.73       454
   macro avg       0.58      0.59      0.57       454
weighted avg       0.84      0.73      0.77       454



C:\Users\mingf\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mingf\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mingf\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Temperature =1

In [15]:
Requests = []
tempr = 1
for index, row in resp.iterrows():
    #print(index)
    scoring_prompt_resp = scoring_prompt+'\n'+'RESPONSE:'+str(row[item])+'. ->'
    Requests.append(
        Request(
            custom_id="item_"+item+"resp_"+str(index),
            params=MessageCreateParamsNonStreaming(
            model="claude-3-7-sonnet-20250219",
            temperature=tempr,
            max_tokens=1024,
            system=role,
            messages=[
                            {"role": "user", "content": scoring_prompt_resp}]
                )
            )
        )
           

In [20]:
Batch1 = cl_client.messages.batches.create(requests=Requests)
Batch2 = cl_client.messages.batches.create(requests=Requests)
Batch3 = cl_client.messages.batches.create(requests=Requests)
Batch4 = cl_client.messages.batches.create(requests=Requests)
Batch5 = cl_client.messages.batches.create(requests=Requests)

In [21]:
Rrsspp1 = cl_client.messages.batches.results(Batch1.id)
Rrsspp2 = cl_client.messages.batches.results(Batch2.id)
Rrsspp3 = cl_client.messages.batches.results(Batch3.id)
Rrsspp4 = cl_client.messages.batches.results(Batch4.id)
Rrsspp5 = cl_client.messages.batches.results(Batch5.id)

Output0 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in Rrsspp1]
Output1 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in Rrsspp2]
Output2 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in Rrsspp3]
Output3 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in Rrsspp4]
Output4 = [ot.to_dict()['result']['message']['content'][0]['text'] for ot in Rrsspp5]

In [22]:
scD_1 = pd.DataFrame({'output_0':Output0, 'output_1':Output1,'output_2':Output2,'output_3':Output3,'output_4':Output4})

scD_1 = pd.concat([scD_1,scD_1.apply(lambda col: [extract_first_number(i) for i in col]).\
rename(columns=dict(zip(['output_'+str(i )for i in range(nrep)],['score_'+str(i )for i in range(nrep)])))], axis=1)

scD_1['mode'] = scD_1.iloc[:,nrep:(2*nrep)].apply(find_mode,axis=1)

scD_1.to_csv(path+'results/'+item+'_claude_tempr'+str(1)+'.csv')

In [9]:
scD_t1 = pd.concat([scD_1,resp.Final],axis=1).dropna()
print(f"QWK: {cohen_kappa_score(scD_t1['mode'],scD_t1['Final'],weights='quadratic')}")
print(classification_report(scD_t1['mode'],scD_t1['Final']))

QWK: 0.23003110256539072
              precision    recall  f1-score   support

           0       0.91      0.17      0.29       240
           1       0.39      0.66      0.49       189
           2       0.14      0.37      0.20        38

    accuracy                           0.39       467
   macro avg       0.48      0.40      0.33       467
weighted avg       0.64      0.39      0.37       467

